<a href="https://colab.research.google.com/github/ThijsVercammen/Masterproef/blob/main/ResNet50_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instaleer nodige tools

In [ ]:
!pip install onnxruntime
!wget https://machinelearningmastery.com/wp-content/uploads/2019/03/zebra.jpg

     |████████████████████████████████| 4.9 MB 21.5 MB/s 
--2022-01-10 15:14:36--  https://machinelearningmastery.com/wp-content/uploads/2019/03/zebra.jpg
Resolving machinelearningmastery.com (machinelearningmastery.com)... 104.26.1.148, 172.67.72.46, 104.26.0.148, ...
Connecting to machinelearningmastery.com (machinelearningmastery.com)|104.26.1.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64972 (63K) [image/jpeg]
Saving to: ‘zebra.jpg’

zebra.jpg           100%[===================>]  63.45K  --.-KB/s    in 0.001s  

2022-01-10 15:14:37 (65.0 MB/s) - ‘zebra.jpg’ saved [64972/64972]



Importeer nodige bibliotheken

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
#from torch.nn import functional as F
import torch.optim as optim
import os
from torch.optim import lr_scheduler
from torch.utils.mobile_optimizer import optimize_for_mobile
from torch.autograd import Variable
from torch.profiler import profile, record_function, ProfilerActivity
import torchvision.transforms.functional as F
import torch.onnx
import onnxruntime as ort
from numpy.testing._private.utils import print_assert_equal
from torchvision import datasets

from __future__ import print_function, division

import numpy as np
import time
import copy

import datetime
from datetime import datetime

plt.ion() 

Inladen van ResNet50 model en convertie naar mobiel model

In [ ]:
model = models.resnet50(pretrained=True)
torch.save(model, "model.pth")
model.eval()
example = torch.rand(1, 3, 416, 416)
traced_script_module = torch.jit.trace(model, example)
traced_script_module.save("model_t2.pt")
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
traced_script_module_optimized._save_for_lite_interpreter("./model_s.pt")

traced_script_module

ResNet(
  original_name=ResNet
  (conv1): Conv2d(original_name=Conv2d)
  (bn1): BatchNorm2d(original_name=BatchNorm2d)
  (relu): ReLU(original_name=ReLU)
  (maxpool): MaxPool2d(original_name=MaxPool2d)
  (layer1): Sequential(
    original_name=Sequential
    (0): Bottleneck(
      original_name=Bottleneck
      (conv1): Conv2d(original_name=Conv2d)
      (bn1): BatchNorm2d(original_name=BatchNorm2d)
      (conv2): Conv2d(original_name=Conv2d)
      (bn2): BatchNorm2d(original_name=BatchNorm2d)
      (conv3): Conv2d(original_name=Conv2d)
      (bn3): BatchNorm2d(original_name=BatchNorm2d)
      (relu): ReLU(original_name=ReLU)
      (downsample): Sequential(
        original_name=Sequential
        (0): Conv2d(original_name=Conv2d)
        (1): BatchNorm2d(original_name=BatchNorm2d)
      )
    )
    (1): Bottleneck(
      original_name=Bottleneck
      (conv1): Conv2d(original_name=Conv2d)
      (bn1): BatchNorm2d(original_name=BatchNorm2d)
      (conv2): Conv2d(original_name=Conv2d)
 

voer model uit op Testafbeelding

In [ ]:
model.eval()
def predict_image(model, image_path, onnx):
    data_transforms = transforms.Compose([
        transforms.Resize(416),
        transforms.CenterCrop(416),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path)
    image_tensor = data_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor.cuda()

    # Turn the input into a Variable
 
    input = Variable(image_tensor)

    # Predict the class of the image
    #now = datetime.now()
    if onnx == False:
        output = model(input)
        preds = np.argmax(output[0].detach().numpy())
    else:
        output = model.run(None,{"input": np.array(input)})
        preds = np.argmax(output[0])  
    return preds

index = predict_image(model,"/content/zebra.jpg")
print(int(index))


340


Converteer naar ONNX

In [ ]:
torch.onnx.export(model,               # model 
                  example,             # model input
                  "model_py.onnx",     # waar opslaan
                  #export_params=True,        # slaag parameters op in model file
                  input_names = ['input'],   # input namen
                  output_names = ['output']) # output namen

Voer ONNX model uit met testafbeelding

In [ ]:
im = Image.open("/content/zebra.jpg")
res_im = im.resize((416, 416))
convert_tensor = transforms.ToTensor()

a = convert_tensor(res_im)
img = F.convert_image_dtype(a, torch.float)
b = img.unsqueeze(0)
input_data = np.asarray(b, dtype=np.float32)

ort_session = ort.InferenceSession("model_py.onnx")
now = datetime.now()

outputs = ort_session.run(
    None,
    {"input": input_data},
)
print(datetime.now()-now)
l = np.argmax(outputs[0])
print(l)


0:00:00.122753
340


Downloaden van Test dataset

In [ ]:
!wget https://s3-us-west-2.amazonaws.com/imagenetv2public/imagenetv2-matched-frequency.tar.gz
!tar -xvf "/content/imagenetv2-matched-frequency.tar.gz"   #[run this cell to extract tar.gz files]

Streaminguitvoer ingekort tot de laatste 5000 regels.
imagenetv2-matched-frequency-format-val/741/08c92c11e5eea6ae8878c9b0dac147cbade36b89.jpeg
imagenetv2-matched-frequency-format-val/741/a5fc5600d9bb68a47e97fee7f419ea48d28a2def.jpeg
imagenetv2-matched-frequency-format-val/741/f7dd021ac3422a21eba0dd5ab7271908b6d1be81.jpeg
imagenetv2-matched-frequency-format-val/741/6866d19851073898a27dac64d7788da0cbe6675f.jpeg
imagenetv2-matched-frequency-format-val/741/d0ee74a2d04dc7f00d5ecb5b38fba98d889ead73.jpeg
imagenetv2-matched-frequency-format-val/741/8eba03604f089f0dde8ae5b5b1649d6da1cc4cdc.jpeg
imagenetv2-matched-frequency-format-val/741/312fa089793eade0ea715e939da1441a39485358.jpeg
imagenetv2-matched-frequency-format-val/741/31fdc4e5b756b2c458aa9c4c8b8993ed54e27aac.jpeg
imagenetv2-matched-frequency-format-val/741/6f01de4490e6de96b6f861e5e573b4a1592324bd.jpeg
imagenetv2-matched-frequency-format-val/741/9179c698319a11a48969adb07311f6f98459d301.jpeg
imagenetv2-matched-frequency-format-val/88/2a4

Evalueer stadaard, mobiel en ONNX model met test dataset

In [ ]:
test_ds = datasets.ImageFolder(root="imagenetv2-matched-frequency-format-val")
val1 = 0
val2 = 0
val3 = 0

convert_tensor = transforms.ToTensor()
ort_session = ort.InferenceSession("model_py.onnx")

for i in range(len(test_ds.imgs)):
  img = test_ds.imgs[i][0]

  res1 = predict_image(model, img, False)
  res2 = predict_image(traced_script_module_optimized, img, False)
  res3 = predict_image(ort_session, img, True)
  t = test_ds.targets[i]
  print("{}".format(i))
  if int(test_ds.classes[t]) == res1:
    val1 += 1
  if int(test_ds.classes[t]) == res2:
    val2 += 1
  if int(test_ds.classes[t]) == res3:
    val3 += 1
print("Standaard model precisie:  {}".format(val1))
print("Mobiel model precisie:     {}".format(val2))
print("ONNX model precisie:       {}".format(val3))